In [1]:
# Import the dependencies.
import pandas as pd
import matplotlib.pyplot as plt
import numpy as np
from datetime import datetime

In [2]:
# Create a set of random latitude and longitude combinations.
lats = np.random.uniform(low=-90.000, high=90.000, size=1500)
lngs = np.random.uniform(low=-180.000, high=180.000, size=1500)
lat_lngs = zip(lats, lngs)
lat_lngs

In [3]:
# Add the latitudes and longitudes to a list.
# Note you can only unzip once# Use the citipy module to determine city based on latitude and longitude.
coordinates = list(lat_lngs)

In [4]:
# Use the citipy module to determine city based on latitude and longitude.
from citipy import citipy

# Create a list for holding the cities.
cities = []
# Identify the nearest city for each latitude and longitude combination by looping through our list
for coordinate in coordinates:
    city = citipy.nearest_city(coordinate[0], coordinate[1]).city_name

    # If the city is unique, then we will add it to the cities list.
    if city not in cities:
        cities.append(city)
# Print the city count to confirm sufficient count.
len(cities)

610

In [5]:
# Import the requests library to be able to call the API
import requests

# Import the API key from our config.py file 
from config import weather_api_key

# Starting URL for Weather Map API Call.
# when making the call we need to adhere to the format specified by the site
# i.e. https://openweathermap.org/current
# i.e. one of the specifications is unit which can be imperial, metric or standard 
url = "http://api.openweathermap.org/data/2.5/weather?units=Imperial&APPID=" + weather_api_key

# Create an endpoint URL for a city based on the format specified 
city_url = url + "&q=" + "Boston"
city_weather = requests.get(city_url)
city_weather.json()


{'coord': {'lon': -71.0598, 'lat': 42.3584},
 'weather': [{'id': 804,
   'main': 'Clouds',
   'description': 'overcast clouds',
   'icon': '04n'}],
 'base': 'stations',
 'main': {'temp': 41.04,
  'feels_like': 33.49,
  'temp_min': 39,
  'temp_max': 43,
  'pressure': 1022,
  'humidity': 81},
 'visibility': 10000,
 'wind': {'speed': 8.05, 'deg': 120},
 'clouds': {'all': 90},
 'dt': 1616920549,
 'sys': {'type': 1,
  'id': 3486,
  'country': 'US',
  'sunrise': 1616927586,
  'sunset': 1616972705},
 'timezone': -14400,
 'id': 4930956,
 'name': 'Boston',
 'cod': 200}

In [6]:
# Get the JSON data by placing the JSON data into a variable 
boston_data = city_weather.json()

In [7]:
# we can then use the keys to obtain the values we want
lat = boston_data["coord"]["lat"]
lng = boston_data["coord"]["lon"]
max_temp = boston_data["main"]["temp_max"]
humidity = boston_data["main"]["humidity"]
clouds = boston_data["clouds"]["all"]
wind = boston_data["wind"]["speed"]
print(lat, lng, max_temp, humidity, clouds, wind)

42.3584 -71.0598 43 81 90 8.05


In [8]:
# Create an empty list to hold the weather data.
city_data = []
# Print the beginning of the logging.
print("Beginning Data Retrieval     ")
print("-----------------------------")

# Create counters.
record_count = 1
set_count = 1
# initialized at 1 since we want the first iteration of the logging for each recorded response and the set to start at 1.

# Loop through all the cities in the list.
# we need to iterate through our list of cities and begin building the URL for each city, while grouping our records in sets of 50. 
# To do this, use for i in range(len(cities)) and the index to tell us when we get to 50. 
# We can also retrieve the city from the cities list and add it to the city_url by using indexing
# We create the for loop with the enumerate() method and reference the index and the city in the list.
for i, city in enumerate(cities):
    # Group cities in sets of 50 for logging purposes.
    # In the conditional statement, we check 
        # if the remainder of the index divided by 50 is equal to 0 and if the index is greater than or equal to 50. 
            # If the statement is true, then the set_count and the record_count are incremented by 1
    if (i % 50 == 0 and i >= 50):
        set_count += 1
        record_count = 1
    # Create endpoint URL with each city.
    # we create the URL endpoint for each city, as before. 
    # However, we are removing the blank spaces in the city name and concatenating the city name with, city.replace(" ","+"). 
    # This will find the corresponding weather data for the city instead of finding the weather data for the first part of the city name
    city_url = url + "&q=" + city.replace(" ","+")
    
    # Log the URL, record, and set numbers and the city.
    # we add a print statement that tells us the record count and set count, and the city that is being processed.
    print(f"Processing Record {record_count} of Set {set_count} | {city}")
    # Add 1 to the record count before the next city is processed.
    record_count += 1
    
    # Run an API request for each of the cities.
    try:
        # Parse the JSON and retrieve data.
        # We parse the JSON file for the current city.
        # If there is weather data for the city, 
            # we will retrieve the latitude, longitude, maximum temperature, humidity, cloudiness, wind speed, and date 
            # and assign those values to variables
        city_weather = requests.get(city_url).json()
        # Parse out the needed data.
        city_lat = city_weather["coord"]["lat"]
        city_lng = city_weather["coord"]["lon"]
        city_max_temp = city_weather["main"]["temp_max"]
        city_humidity = city_weather["main"]["humidity"]
        city_clouds = city_weather["clouds"]["all"]
        city_wind = city_weather["wind"]["speed"]
        city_country = city_weather["sys"]["country"]
        # Convert the date to ISO standard.
        city_date = datetime.utcfromtimestamp(city_weather["dt"]).strftime('%Y-%m-%d %H:%M:%S')
        # Append the city information into city_data list.
        # We append the cities list with a dictionary for that city, where the key-value pairs are the values from our weather parameters
        city_data.append({"City": city.title(),
                          "Lat": city_lat,
                          "Lng": city_lng,
                          "Max Temp": city_max_temp,
                          "Humidity": city_humidity,
                          "Cloudiness": city_clouds,
                          "Wind Speed": city_wind,
                          "Country": city_country,
                          "Date": city_date})

# If an error is experienced, skip the city.
    # If there is no weather data for the city, i.e. , a <Response [404]> then there is no weather to retrieve 
    # City not found. Skipping... is printed.
    # pass statement, which is a general purpose statement to handle all errors encountered and to allow the program to continue
    except:
        print("City not found. Skipping...")
        pass

# Indicate that Data Loading is complete.
# closing print statement, which will let us know the data retrieval has been completed
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Beginning Data Retrieval     
-----------------------------
Processing Record 1 of Set 1 | westport
Processing Record 2 of Set 1 | barrow
Processing Record 3 of Set 1 | karratha
Processing Record 4 of Set 1 | saskylakh
Processing Record 5 of Set 1 | ushuaia
Processing Record 6 of Set 1 | kodiak
Processing Record 7 of Set 1 | punta arenas
Processing Record 8 of Set 1 | bluff
Processing Record 9 of Set 1 | taolanaro
City not found. Skipping...
Processing Record 10 of Set 1 | kaitangata
Processing Record 11 of Set 1 | rikitea
Processing Record 12 of Set 1 | ilulissat
Processing Record 13 of Set 1 | comodoro rivadavia
Processing Record 14 of Set 1 | manzanillo
Processing Record 15 of Set 1 | concepcion del oro
Processing Record 16 of Set 1 | berlevag
Processing Record 17 of Set 1 | the valley
Processing Record 18 of Set 1 | esperance
Processing Record 19 of Set 1 | port elizabeth
Processing Record 20 of Set 1 | yongan
Processing Record 21 of Set 1 | balimo
City not found. Skipping...
Proce

Processing Record 38 of Set 4 | wyndham
Processing Record 39 of Set 4 | bolungarvik
City not found. Skipping...
Processing Record 40 of Set 4 | mys shmidta
City not found. Skipping...
Processing Record 41 of Set 4 | new waterford
Processing Record 42 of Set 4 | kavieng
Processing Record 43 of Set 4 | batagay
Processing Record 44 of Set 4 | oliver
Processing Record 45 of Set 4 | camacha
Processing Record 46 of Set 4 | bilibino
Processing Record 47 of Set 4 | king city
Processing Record 48 of Set 4 | tual
Processing Record 49 of Set 4 | panguna
Processing Record 50 of Set 4 | ondorhaan
City not found. Skipping...
Processing Record 1 of Set 5 | nabire
Processing Record 2 of Set 5 | mogochin
City not found. Skipping...
Processing Record 3 of Set 5 | tezu
Processing Record 4 of Set 5 | malmyzh
Processing Record 5 of Set 5 | gloria do goita
Processing Record 6 of Set 5 | cockburn town
Processing Record 7 of Set 5 | casalpusterlengo
Processing Record 8 of Set 5 | clyde river
Processing Record

Processing Record 26 of Set 8 | callaway
Processing Record 27 of Set 8 | namtsy
Processing Record 28 of Set 8 | san patricio
Processing Record 29 of Set 8 | changping
Processing Record 30 of Set 8 | marica
Processing Record 31 of Set 8 | kavaratti
Processing Record 32 of Set 8 | atambua
Processing Record 33 of Set 8 | tabiauea
City not found. Skipping...
Processing Record 34 of Set 8 | bontang
Processing Record 35 of Set 8 | porto novo
Processing Record 36 of Set 8 | kunnamangalam
Processing Record 37 of Set 8 | espanola
Processing Record 38 of Set 8 | jatai
Processing Record 39 of Set 8 | orange cove
Processing Record 40 of Set 8 | rawson
Processing Record 41 of Set 8 | chagda
City not found. Skipping...
Processing Record 42 of Set 8 | posse
Processing Record 43 of Set 8 | caravelas
Processing Record 44 of Set 8 | longyearbyen
Processing Record 45 of Set 8 | krasnoselkup
Processing Record 46 of Set 8 | tomatlan
Processing Record 47 of Set 8 | port macquarie
Processing Record 48 of Set

Processing Record 16 of Set 12 | brae
Processing Record 17 of Set 12 | umzimvubu
City not found. Skipping...
Processing Record 18 of Set 12 | kaset sombun
Processing Record 19 of Set 12 | tonantins
Processing Record 20 of Set 12 | acu
Processing Record 21 of Set 12 | itarema
Processing Record 22 of Set 12 | turbat
Processing Record 23 of Set 12 | chapais
Processing Record 24 of Set 12 | keuruu
Processing Record 25 of Set 12 | batagay-alyta
Processing Record 26 of Set 12 | arkhangelsk
Processing Record 27 of Set 12 | naifaru
Processing Record 28 of Set 12 | khonuu
City not found. Skipping...
Processing Record 29 of Set 12 | zyryanka
Processing Record 30 of Set 12 | port hedland
Processing Record 31 of Set 12 | dasoguz
Processing Record 32 of Set 12 | warqla
City not found. Skipping...
Processing Record 33 of Set 12 | bhawana
Processing Record 34 of Set 12 | sao filipe
Processing Record 35 of Set 12 | muros
Processing Record 36 of Set 12 | morro bay
Processing Record 37 of Set 12 | san f

In [9]:
len_city_data = len(city_data)
len_city_data

561

In [10]:
# Convert the array of dictionaries to a Pandas DataFrame.
city_data_df = pd.DataFrame(city_data)
city_data_df.head(10)

,City,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Country,Date
0,Westport,41.1415,-73.3579,46.00,87,1,9.22,US,2021-03-28 08:42:04
1,Barrow,71.2906,-156.7887,-20.20,76,1,8.05,US,2021-03-28 08:39:55
2,Karratha,-20.7377,116.8463,89.98,51,56,13.87,AU,2021-03-28 08:47:25
3,Saskylakh,71.9167,114.0833,4.17,93,99,2.13,RU,2021-03-28 08:39:56
4,Ushuaia,-54.8000,-68.3000,42.80,93,75,46.04,AR,2021-03-28 08:37:33
5,Kodiak,57.7900,-152.4072,33.80,44,1,21.85,US,2021-03-28 08:40:44
6,Punta Arenas,-53.1500,-70.9167,44.60,61,40,29.93,CL,2021-03-28 08:35:39
7,Bluff,-46.6000,168.3333,54.00,95,100,5.01,NZ,2021-03-28 08:39:56
8,Kaitangata,-46.2817,169.8464,54.21,95,85,3.42,NZ,2021-03-28 08:40:45
9,Rikitea,-23.1203,-134.9692,78.89,75,2,17.45,PF,2021-03-28 08:36:21


In [11]:
new_column_order = ['City','Country','Date','Lat','Lng','Max Temp','Humidity','Cloudiness','Wind Speed']
city_data_df[new_column_order]

,City,Country,Date,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed
0,Westport,US,2021-03-28 08:42:04,41.1415,-73.3579,46.00,87,1,9.22
1,Barrow,US,2021-03-28 08:39:55,71.2906,-156.7887,-20.20,76,1,8.05
2,Karratha,AU,2021-03-28 08:47:25,-20.7377,116.8463,89.98,51,56,13.87
3,Saskylakh,RU,2021-03-28 08:39:56,71.9167,114.0833,4.17,93,99,2.13
4,Ushuaia,AR,2021-03-28 08:37:33,-54.8000,-68.3000,42.80,93,75,46.04
...,...,...,...,...,...,...,...,...,...
556,Margate,GB,2021-03-28 08:42:09,51.3813,1.3862,52.00,65,100,5.01
557,Dhidhdhoo,MV,2021-03-28 08:49:34,6.8833,73.1000,83.59,73,17,8.68
558,Maryville,US,2021-03-28 08:49:34,35.7565,-83.9705,63.00,100,75,3.44
559,Itum-Kale,RU,2021-03-28 08:49:35,42.7356,45.5758,39.87,82,99,6.26


In [12]:
# Create the output file (CSV).
output_data_file = "weather_data/cities.csv"
# Export the City_Data into a CSV.
# The last line in the code block will export the DataFrame to a CSV file, with the index label (or column A) header as "City_ID." 
# If we ever need to export the CSV file to a DataFrame, that header will be present in the DataFrame
city_data_df.to_csv(output_data_file, index_label="City_ID")
